<a href="https://colab.research.google.com/github/spsagar13/Natural-Language-Processing/blob/main/NLP_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

In [ ]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [ ]:
train_data, test_data = imdb['train'], imdb['test']


training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(str(l.numpy()))

for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [ ]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

       b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all '
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what wa

In [ ]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),

                                    tf.keras.layers.Flatten(),
                                    # tf.keras.layers.GlobalAveragePooling1D(),
                                    tf.keras.layers.Dense(6, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10

training_labels_final = np.array(training_labels, dtype=float)
testing_labels_final = np.array(testing_labels, dtype=float)

model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 5ms/step - loss: 0.6136 - accuracy: 0.6230 - val_loss: 0.3533 - val_accuracy: 0.8448
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2483 - accuracy: 0.9057 - val_loss: 0.3673 - val_accuracy: 0.8402
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1091 - accuracy: 0.9748 - val_loss: 0.4456 - val_accuracy: 0.8286
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0321 - accuracy: 0.9962 - val_loss: 0.5149 - val_accuracy: 0.8276
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0150 - accuracy: 0.9980 - val_loss: 0.5757 - val_accuracy: 0.8268
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0073 - accuracy: 0.9990 - val_loss: 0.6265 - val_accuracy: 0.8214
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0032 - accuracy: 0.9995 - val_loss: 0.7101 - val_accuracy: 0.8150
Epoch 

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word +"\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.collab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [ ]:
#upload the file to http://projector.tensorflow.org/ to visualize

In [ ]:
sentence = "Isnt this amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 68, 105, 12, 7, 491, 1216]]
